# HappyRobot Load Management API Testing

This notebook provides examples of how to interact with the HappyRobot backend API for load management.

## Features:
- Create new loads
- Update existing loads
- Assign loads (set status to agreed)
- List all loads
- Get specific load details
- Delete loads

## Prerequisites:
- Backend running on port 8000
- Valid API key (if authentication is enabled)


In [2]:
# Import required libraries
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
from typing import Dict, Any, Optional
import uuid


## Configuration

Set your backend URL and API key here:


In [3]:
# Configuration
BASE_URL = "http://localhost:8000"
BASE_URL = "http://happyrobot-1700442240.eu-north-1.elb.amazonaws.com"  # Change this to your backend URL
API_KEY = "HapRob-OTVHhErcXLu2eKkUMP6lDtrd8UNi61KZo4FvGALqem0NoJO1uWlz7OywCN0BNoNaG2x5Y"  # Default API key that works

# Headers for API requests
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "X-API-Key": API_KEY  # API key header
}

print(f"Backend URL: {BASE_URL}")
print(f"API Key: {API_KEY[:10]}..." if len(API_KEY) > 10 else f"API Key: {API_KEY}")


Backend URL: http://happyrobot-1700442240.eu-north-1.elb.amazonaws.com
API Key: HapRob-OTV...


## Helper Functions


In [4]:
def make_request(method: str, endpoint: str, data: Optional[Dict] = None) -> Dict[str, Any]:
    """
    Make an API request to the backend
    
    Args:
        method: HTTP method (GET, POST, PATCH, DELETE)
        endpoint: API endpoint (e.g., '/shipments')
        data: Request data for POST/PATCH requests
    
    Returns:
        Response data as dictionary
    """
    url = f"{BASE_URL}{endpoint}"
    
    try:
        if method.upper() == "GET":
            response = requests.get(url, headers=headers)
        elif method.upper() == "POST":
            response = requests.post(url, headers=headers, json=data)
        elif method.upper() == "PATCH":
            response = requests.patch(url, headers=headers, json=data)
        elif method.upper() == "DELETE":
            response = requests.delete(url, headers=headers)
        else:
            raise ValueError(f"Unsupported HTTP method: {method}")
        
        # Check if request was successful
        if response.status_code in [200, 201, 204]:
            if response.status_code == 204:  # No content
                return {"success": True, "message": "Operation completed successfully"}
            return {"success": True, "data": response.json()}
        else:
            return {
                "success": False, 
                "error": f"HTTP {response.status_code}",
                "message": response.text
            }
            
    except requests.exceptions.RequestException as e:
        return {"success": False, "error": "Request failed", "message": str(e)}

def health_check() -> bool:
    """Check if the backend is healthy"""
    result = make_request("GET", "/health")
    if result["success"]:
        print("✅ Backend is healthy")
        return True
    else:
        print(f"❌ Backend health check failed: {result.get('message', 'Unknown error')}")
        return False

def print_response(result: Dict[str, Any], title: str = "Response"):
    """Pretty print API response"""
    print(f"\n{'='*50}")
    print(f"{title}")
    print(f"{'='*50}")
    
    if result["success"]:
        print("✅ Success!")
        if "data" in result:
            print(json.dumps(result["data"], indent=2, default=str))
        elif "message" in result:
            print(result["message"])
    else:
        print("❌ Failed!")
        print(f"Error: {result.get('error', 'Unknown error')}")
        print(f"Message: {result.get('message', 'No message')}")
    print(f"{'='*50}\n")


## 1. Health Check

First, let's verify that the backend is running and accessible:


In [5]:
# Check backend health
health_check()


✅ Backend is healthy


True

## 2. Create a New Load

Create a new load with all the required and optional fields:


In [6]:
# Create a new load
def create_load(load_data: Dict[str, Any]) -> Dict[str, Any]:
    """Create a new load"""
    return make_request("POST", "/shipments", load_data)
print(API_KEY)
# Example load data
new_load = {
    "load_id": f"LD-NOTEBOOK-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    "origin": "Los Angeles, CA",
    "destination": "New York, NY",
    "pickup_datetime": (datetime.now() + timedelta(days=1)).isoformat() + "Z",
    "delivery_datetime": (datetime.now() + timedelta(days=3)).isoformat() + "Z",
    "equipment_type": "Dry Van",
    "loadboard_rate": 2500.50,
    "weight": 1500.0,
    "commodity_type": "Electronics",
    "num_of_pieces": 100,
    "miles": 2800.0,
    "dimensions": "48x40x60 in",
    "notes": "Created from Jupyter notebook - handle with care"
}

print("Creating new load...")
print(f"Load ID: {new_load['load_id']}")
print(f"Route: {new_load['origin']} → {new_load['destination']}")

result = create_load(new_load)
print_response(result, "Create Load")


HapRob-OTVHhErcXLu2eKkUMP6lDtrd8UNi61KZo4FvGALqem0NoJO1uWlz7OywCN0BNoNaG2x5Y
Creating new load...
Load ID: LD-NOTEBOOK-20250911-210625
Route: Los Angeles, CA → New York, NY

Create Load
✅ Success!
{
  "load_id": "LD-NOTEBOOK-20250911-210625",
  "origin": "Los Angeles, CA",
  "destination": "New York, NY",
  "pickup_datetime": "2025-09-12T21:06:25.418701Z",
  "delivery_datetime": "2025-09-14T21:06:25.418701Z",
  "equipment_type": "Dry Van",
  "loadboard_rate": 2500.5,
  "notes": "Created from Jupyter notebook - handle with care",
  "weight": 1500.0,
  "commodity_type": "Electronics",
  "num_of_pieces": 100,
  "miles": 2800.0,
  "dimensions": "48x40x60 in",
  "agreed_price": null,
  "carrier_description": null,
  "assigned_via_url": true,
  "time_per_call_seconds": null,
  "avg_time_per_call_seconds": null,
  "status": "pending",
  "id": "c6edd05a-dfed-4b2f-bd97-02aacadfe6a6",
  "created_at": "2025-09-11T19:06:25.812958",
  "updated_at": "2025-09-11T19:06:25.812960"
}



## 3. Assign a Load (Set Status to Agreed)

This is the specific operation you requested - changing a load status to "assigned" (agreed) with the required agreed_price and carrier_description:


In [14]:
# First, let's get all loads to see what we have
def get_all_loads() -> Dict[str, Any]:
    """Get all loads"""
    return make_request("GET", "/shipments")

# Get all loads
all_loads_result = get_all_loads()
print_response(all_loads_result, "All Loads")

# Find a load to assign (preferably one that's not already assigned)
if all_loads_result["success"] and all_loads_result["data"]:
    loads = all_loads_result["data"]
    
    # Find a pending load to assign
    pending_loads = [load for load in loads if load["status"] == "pending"]
    
    if pending_loads:
        load_to_assign = pending_loads[0]
        print(f"\n🎯 Found load to assign: {load_to_assign['load_id']}")
        print(f"   Route: {load_to_assign['origin']} → {load_to_assign['destination']}")
        print(f"   Current status: {load_to_assign['status']}")
        
        # Assign the load with required fields
        assign_data = {
            "status": "agreed",
            "agreed_price": 2800.00,  # Required when status is agreed
            "carrier_description": "ABC Transport Co."  # Required when status is agreed
        }
        
        print(f"\n📝 Assigning load with:")
        print(f"   Agreed Price: ${assign_data['agreed_price']}")
        print(f"   Carrier: {assign_data['carrier_description']}")
        
        # Make the PATCH request to assign the load
        assign_result = make_request("PATCH", f"/shipments/{load_to_assign['id']}", assign_data)
        print_response(assign_result, "Assign Load")
        
    else:
        print("\n⚠️ No pending loads found to assign")
        print("All loads are already assigned or no loads exist")
        
        # Show current loads status
        if loads:
            print("\n📊 Current loads status:")
            for load in loads:
                print(f"   {load['load_id']}: {load['status']}")
else:
    print("❌ Could not retrieve loads")



All Loads
✅ Success!
[
  {
    "load_id": "LD-NOTEBOOK-20250911-210625",
    "origin": "Los Angeles, CA",
    "destination": "New York, NY",
    "pickup_datetime": "2025-09-12T21:06:25.418701Z",
    "delivery_datetime": "2025-09-14T21:06:25.418701Z",
    "equipment_type": "Dry Van",
    "loadboard_rate": 2500.5,
    "notes": "Created from Jupyter notebook - handle with care",
    "weight": 1500.0,
    "commodity_type": "Electronics",
    "num_of_pieces": 100,
    "miles": 2800.0,
    "dimensions": "48x40x60 in",
    "agreed_price": 2800.0,
    "carrier_description": "ABC Transport Co.",
    "assigned_via_url": true,
    "time_per_call_seconds": null,
    "avg_time_per_call_seconds": 30.0,
    "status": "agreed",
    "id": "c6edd05a-dfed-4b2f-bd97-02aacadfe6a6",
    "created_at": "2025-09-11T19:06:25.812958",
    "updated_at": "2025-09-11T19:06:26.060657"
  },
  {
    "load_id": "LD-2025-0008",
    "origin": "Hamburg",
    "destination": "Stockholm",
    "pickup_datetime": "2025-09-25T

In [8]:
# First, let's verify the load exists and check its current status
load_id = "LD-2025-0008"

print(f"🔍 Checking if load {load_id} exists...")

# Get all loads to find the one we want
all_loads_result = get_all_loads()

if all_loads_result["success"] and all_loads_result["data"]:
    loads = all_loads_result["data"]
    
    # Find the specific load
    target_load = None
    for load in loads:
        if load["load_id"] == load_id:
            target_load = load
            break
    
    if target_load:
        print(f"✅ Found load: {target_load['load_id']}")
        print(f"   Current status: {target_load['status']}")
        print(f"   Route: {target_load['origin']} → {target_load['destination']}")
        print(f"   Loadboard rate: ${target_load.get('loadboard_rate', 'N/A')}")
        
        if target_load['status'] == 'pending':
            print(f"\n🎯 Load is pending - ready to assign!")
            
            # Now assign the load
            print(f"\n🚀 Assigning load {load_id}...")
            
            url = f"http://localhost:8000/shipments/{target_load['id']}"
            headers = {
                "Content-Type": "application/json",
                "X-API-Key": "HapRob-OTVHhErcXLu2eKkUMP6lDtrd8UNi61KZo4FvGALqem0NoJO1uWlz7OywCN0BNoNaG2x5Y"
            }
            data = {
                "status": "agreed",
                "agreed_price": 1700.0,
                "carrier_description": "ABC Transport Co.",
                "time_per_call_seconds": 120
            }
            
            try:
                response = requests.patch(url, headers=headers, json=data)
                
                print(f"📊 Response Status Code: {response.status_code}")
                
                if response.status_code == 200:
                    print("\n✅ Success! Load assigned successfully:")
                    updated_load = response.json()
                    print(f"   New status: {updated_load['status']}")
                    print(f"   Agreed price: ${updated_load.get('agreed_price', 'N/A')}")
                    print(f"   Carrier: {updated_load.get('carrier_description', 'N/A')}")
                else:
                    print(f"\n❌ Request failed with status code: {response.status_code}")
                    print(f"Response: {response.text}")
                    
            except requests.exceptions.RequestException as e:
                print(f"\n💥 Request failed: {str(e)}")
                
        elif target_load['status'] == 'agreed':
            print(f"\n⚠️ Load is already assigned (agreed)")
            print(f"   Agreed price: ${target_load.get('agreed_price', 'N/A')}")
            print(f"   Carrier: {target_load.get('carrier_description', 'N/A')}")
        else:
            print(f"\n❓ Load has unexpected status: {target_load['status']}")
    else:
        print(f"❌ Load {load_id} not found!")
        print("\n📋 Available loads:")
        for load in loads:
            print(f"   {load['load_id']}: {load['status']}")
else:
    print("❌ Could not retrieve loads from the API")


🔍 Checking if load LD-2025-0008 exists...
✅ Found load: LD-2025-0008
   Current status: pending
   Route: Hamburg → Stockholm
   Loadboard rate: $2800.0

🎯 Load is pending - ready to assign!

🚀 Assigning load LD-2025-0008...

💥 Request failed: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /shipments/8035a8f7-f62c-4f2a-82f2-c231c5630a2d (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002F10A078A60>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))


## 🎲 Test Random Load Endpoint

Let's test the new random load endpoint that can retrieve a random shipment, optionally filtered by origin.


In [9]:
# Test 1: Get a completely random load
print("🎲 Test 1: Getting a completely random load...")
response = requests.get(f"{BASE_URL}/shipments/random", headers=headers)
print(f"📊 Response Status Code: {response.status_code}")

if response.status_code == 200:
    random_load = response.json()
    print(f"✅ Random load retrieved:")
    print(f"   Load ID: {random_load['load_id']}")
    print(f"   Route: {random_load['origin']} → {random_load['destination']}")
    print(f"   Equipment: {random_load['equipment_type']}")
    print(f"   Status: {random_load['status']}")
    print(f"   Loadboard Rate: ${random_load['loadboard_rate']}")
else:
    print(f"❌ Error: {response.text}")

print("\n" + "="*60 + "\n")

🎲 Test 1: Getting a completely random load...
📊 Response Status Code: 200
✅ Random load retrieved:
   Load ID: LD-2025-0001
   Route: Madrid → Paris
   Equipment: Dry Van
   Status: pending
   Loadboard Rate: $1800.5




In [10]:
# Test 2: Get a random load from a specific origin
print("🎯 Test 2: Getting a random load from Madrid...")
response = requests.get(f"{BASE_URL}/shipments/random?origin=Madrid", headers=headers)
print(f"📊 Response Status Code: {response.status_code}")

if response.status_code == 200:
    madrid_load = response.json()
    print(f"✅ Random Madrid load retrieved:")
    print(f"   Load ID: {madrid_load['load_id']}")
    print(f"   Route: {madrid_load['origin']} → {madrid_load['destination']}")
    print(f"   Equipment: {madrid_load['equipment_type']}")
    print(f"   Status: {madrid_load['status']}")
    print(f"   Loadboard Rate: ${madrid_load['loadboard_rate']}")
else:
    print(f"❌ Error: {response.text}")

print("\n" + "="*60 + "\n")

🎯 Test 2: Getting a random load from Madrid...
📊 Response Status Code: 200
✅ Random Madrid load retrieved:
   Load ID: LD-2025-0001
   Route: Madrid → Paris
   Equipment: Dry Van
   Status: pending
   Loadboard Rate: $1800.5




In [11]:

# Test 3: Try to get a random load from a non-existent origin
print("🔍 Test 3: Getting a random load from non-existent origin 'Tokyo'...")
response = requests.get(f"{BASE_URL}/shipments/random?origin=Tokyo", headers=headers)
print(f"📊 Response Status Code: {response.status_code}")

if response.status_code == 404:
    print("✅ Correctly returned 404 for non-existent origin")
    print(f"   Error message: {response.json()['detail']}")
else:
    print(f"❌ Unexpected response: {response.text}")

print("\n" + "="*60 + "\n")

# Test 4: Get multiple random loads to see variety
print("🎲 Test 4: Getting 3 random loads to see variety...")
for i in range(3):
    response = requests.get(f"{BASE_URL}/shipments/random", headers=headers)
    if response.status_code == 200:
        load = response.json()
        print(f"   Random load {i+1}: {load['load_id']} ({load['origin']} → {load['destination']})")
    else:
        print(f"   Error getting random load {i+1}: {response.text}")

print("\n🎉 Random load endpoint testing complete!")


🔍 Test 3: Getting a random load from non-existent origin 'Tokyo'...
📊 Response Status Code: 404
✅ Correctly returned 404 for non-existent origin
   Error message: No pending shipments found with origin containing 'Tokyo'


🎲 Test 4: Getting 3 random loads to see variety...
   Random load 1: LD-2025-0006 (London → Frankfurt)
   Random load 2: LD-2025-0002 (Berlin → Barcelona)
   Random load 3: LD-2025-0005 (Rome → Vienna)

🎉 Random load endpoint testing complete!


## ⏱️ Test Time Per Call Functionality

Let's test the new time per call feature that allows manual input of call times for both manual and API assignments.


In [ ]:
# Test 1: Manual assignment with time per call
print("👤 Test 1: Manual assignment with time per call...")

# First, get a pending load to assign manually
all_loads_result = get_all_loads()
if all_loads_result["success"] and all_loads_result["data"]:
    loads = all_loads_result["data"]
    pending_loads = [load for load in loads if load["status"] == "pending"]
    
    if pending_loads:
        load_to_assign = pending_loads[0]
        print(f"   Assigning load: {load_to_assign['load_id']}")
        print(f"   Route: {load_to_assign['origin']} → {load_to_assign['destination']}")
        
        # Manual assignment with time per call
        manual_assign_data = {
            "status": "agreed",
            "agreed_price": 2500.0,
            "carrier_description": "Manual Carrier Co.",
            "time_per_call_seconds": 180.0  # 3 minutes for manual call
        }
        
        # Use the manual endpoint
        manual_result = make_request("PATCH", f"/shipments/{load_to_assign['id']}/manual", manual_assign_data)
        
        if manual_result["success"]:
            assigned_load = manual_result["data"]
            print(f"✅ Manual assignment successful:")
            print(f"   Status: {assigned_load['status']}")
            print(f"   Assigned via URL: {assigned_load['assigned_via_url']}")
            print(f"   Time per call: {assigned_load['time_per_call_seconds']} seconds")
            print(f"   Avg time per call: {assigned_load['avg_time_per_call_seconds']} seconds")
        else:
            print(f"❌ Manual assignment failed: {manual_result['message']}")
    else:
        print("⚠️ No pending loads available for manual assignment")

print("\n" + "="*60 + "\n")



# Test 3: Check stats with actual time per call values
print("📊 Test 3: Check stats with actual time per call values...")

stats_result = make_request("GET", "/shipments/stats")
if stats_result["success"]:
    stats = stats_result["data"]
    print("✅ Stats retrieved successfully:")
    print(f"   Manual loads:")
    print(f"     Count: {stats['manual']['count']}")
    print(f"     Avg time per call: {stats['manual']['avg_time_per_call_seconds']:.1f} seconds")
    print(f"   API loads:")
    print(f"     Count: {stats['url_api']['count']}")
    print(f"     Avg time per call: {stats['url_api']['avg_time_per_call_seconds']:.1f} seconds")
else:
    print(f"❌ Failed to get stats: {stats_result['message']}")

print("\n🎉 Time per call functionality testing complete!")


👤 Test 1: Manual assignment with time per call...
   Assigning load: LD-2025-0004
   Route: Amsterdam → Milan
✅ Manual assignment successful:
   Status: agreed
   Assigned via URL: False
   Time per call: 180.0 seconds
   Avg time per call: 180.0 seconds


🤖 Test 2: API assignment with time per call...
   Assigning load: LD-2025-0004
   Route: Amsterdam → Milan
❌ API assignment failed: {"detail":"Shipment with id LD-2025-0005 not found"}


📊 Test 3: Check stats with actual time per call values...
✅ Stats retrieved successfully:
   Manual loads:
     Count: 4
     Avg time per call: 180.0 seconds
   API loads:
     Count: 2
     Avg time per call: 0.0 seconds

🎉 Time per call functionality testing complete!


In [18]:
# Test 2: API assignment with time per call
print("🤖 Test 2: API assignment with time per call...")

id_api="LD-2025-0002"
        
# API assignment with time per call
api_assign_data = {
    "status": "agreed",
    "agreed_price": 2200.0,
    "carrier_description": "API Carrier Co.",
    "time_per_call_seconds": 45.0  # 45 seconds for API call
}

# Use the regular API endpoint
api_result = make_request("PATCH", f"/shipments/{id_api}", api_assign_data)

if api_result["success"]:
    assigned_load = api_result["data"]
    print(f"✅ API assignment successful:")
    print(f"   Status: {assigned_load['status']}")
    print(f"   Assigned via URL: {assigned_load['assigned_via_url']}")
    print(f"   Time per call: {assigned_load['time_per_call_seconds']} seconds")
    print(f"   Avg time per call: {assigned_load['avg_time_per_call_seconds']} seconds")
else:
    print(f"❌ API assignment failed: {api_result['message']}")

🤖 Test 2: API assignment with time per call...
❌ API assignment failed: {"detail":"Shipment with id LD-2025-0002 not found"}


In [ ]:
# 🎉 NEW: Test with human-readable load_id format
print("🎉 Testing with human-readable load_id format...")

# Now you can use the human-readable load_id directly!
load_id = "LD-2025-0002"

# API assignment with time per call using human-readable ID
api_assign_data = {
    "status": "agreed",
    "agreed_price": 2200.0,
    "carrier_description": "API Carrier Co. (Human-readable ID)",
    "time_per_call_seconds": 45.0
}

# Use the human-readable load_id directly in the endpoint
api_result = make_request("PATCH", f"/shipments/{load_id}", api_assign_data)

if api_result["success"]:
    assigned_load = api_result["data"]
    print(f"✅ API assignment successful with human-readable ID:")
    print(f"   Load ID: {assigned_load['load_id']}")
    print(f"   Status: {assigned_load['status']}")
    print(f"   Agreed Price: ${assigned_load['agreed_price']}")
    print(f"   Carrier: {assigned_load['carrier_description']}")
    print(f"   Time per call: {assigned_load['time_per_call_seconds']} seconds")
    print(f"   Avg time per call: {assigned_load['avg_time_per_call_seconds']} seconds")
else:
    print(f"❌ API assignment failed: {api_result['message']}")


In [ ]:
# 🧪 Test other endpoints with human-readable IDs
print("🧪 Testing other endpoints with human-readable load_id format...")

# Test GET endpoint
print("\n1️⃣ Testing GET endpoint:")
get_result = make_request("GET", f"/shipments/{load_id}")
if get_result["success"]:
    print(f"✅ GET successful: {get_result['data']['load_id']} - {get_result['data']['origin']} → {get_result['data']['destination']}")
else:
    print(f"❌ GET failed: {get_result['message']}")

# Test with a different load
test_load_id = "LD-2025-0001"
print(f"\n2️⃣ Testing with {test_load_id}:")
get_result2 = make_request("GET", f"/shipments/{test_load_id}")
if get_result2["success"]:
    print(f"✅ GET successful: {get_result2['data']['load_id']} - {get_result2['data']['origin']} → {get_result2['data']['destination']}")
else:
    print(f"❌ GET failed: {get_result2['message']}")

# Test DELETE endpoint (be careful!)
print(f"\n⚠️ DELETE endpoint also works with human-readable IDs")
print("   (Uncomment the next lines to test DELETE - be careful!)")
# delete_result = make_request("DELETE", f"/shipments/{test_load_id}")
# if delete_result["success"]:
#     print(f"✅ DELETE successful")
# else:
#     print(f"❌ DELETE failed: {delete_result['message']}")


# 🎉 Backend Updated: Human-Readable Load IDs Supported!

## ✅ What Changed

The backend now accepts **both formats** in all endpoints:

### 🔧 **Supported ID Formats:**
- **UUID format**: `40fc4084-39c1-447e-971e-4d63006825be` (internal ID)
- **Human-readable format**: `LD-2025-0002` (load_id)

### 📡 **Updated Endpoints:**
- `GET /shipments/{id}` - Get shipment by ID
- `PATCH /shipments/{id}` - Update shipment by ID  
- `PATCH /shipments/{id}/manual` - Manual update by ID
- `DELETE /shipments/{id}` - Delete shipment by ID

### 🚀 **Benefits:**
- **Easier API testing** - Use human-readable IDs like `LD-2025-0002`
- **Better user experience** - No need to look up UUIDs
- **Backward compatible** - Existing UUID-based calls still work
- **Frontend ready** - Frontend automatically works with both formats

### 💡 **Example Usage:**
```bash
# Both of these work now:
curl -X PATCH "http://localhost:8000/shipments/LD-2025-0002" \
  -H "X-API-Key: your-key" \
  -d '{"status": "agreed", "agreed_price": 2200.0}'

curl -X PATCH "http://localhost:8000/shipments/40fc4084-39c1-447e-971e-4d63006825be" \
  -H "X-API-Key: your-key" \
  -d '{"status": "agreed", "agreed_price": 2200.0}'
```
